# Exercise 4.8 Task

## Contents
1. Importing libraries and latest df from 4.7
UPDATE: per recommendation from 4.9, moving data cleaning to this notebook, removing outliers from prices.
2. Found aggregate mean of "order_number" column grouped by "department_id" for entire dataframe.
3. Result analysis - compared to df subset
4. Created 'loyalty_flag' for existing customers
5. Used Loyalty Flag to check the basic statistics of the product prices for each customer loyalty designation
6. Created spending flag for each user based on average price across all their orders
7. Created order frequency flag that marks the regularity of a user's ordering behavior according to the median days in the "days_since_prior_order" column.
8. Code Cleanliness and Markup Check
9. Exported dataframe in pickle format

### Importing Libraries

In [29]:
import pandas as pd
import numpy as np
import os

In [64]:
# Setting path to Achievement 4 master folder
path = r'/Users/jeremyobach/Documents/Data Analytics/CareerFoundry/Achievement 4- Python for DA/DEC23 Instacart Basket Analysis - MASTER FOLDER'
#checking path set correctly
path

'/Users/jeremyobach/Documents/Data Analytics/CareerFoundry/Achievement 4- Python for DA/DEC23 Instacart Basket Analysis - MASTER FOLDER'

## 1. importing ords_prods_merge_derived, latest df version from 4.7, as ords_prods_merge

In [65]:
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data','ords_prods_merge_derived.pkl'))

### UPDATE: Per recommendation from 4.9 notebook to be strictly for data visualization, moving data cleaning to 4.8 notebook

From a failed histogram that scewed all data into one bucket, followd by a scatterplot of prices, we found that there were 99,999 dollar items and 14,900 dollar items in ['prices']. 

Using loc() function to mark these outlier values as nan/missing.

In [66]:
ords_prods_merge.loc[ords_prods_merge['prices'] > 100, 'prices'] = np.nan

In [67]:
ords_prods_merge['prices'].max()

25.0

Perfect. Logically, you know that a 25 dollar grocery item is much more likely than a $14,900 item (and especially a 99,999 dollar item!).

Resuming Exercise 4.8 Task

## 2. Find aggregate mean of "order_number" column grouped by "department_id" for entire dataframe

In [68]:
ords_prods_merge.groupby('department_id').agg({'order_number':['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


## 3. Analyze the result
### Compared to the subset, the results from the entire dataframe are not terribly different. 

Department_id's 1-5, 7 , 9-13, 15, 17-19 would all round to the same integer of order numbers. Department_id's 6,8,14, 16,20, and 21 differ by about 1 order.

## 4. Create a Loyalty flag for existing customers using the transform() and loc() functions

#### 1. Splitting the data into groups based on "user-Id column
#### 2. Applying the transform() function on the "order_number" column to generate the maximum orders for each user
Using numpy to derive max for the column
#### 3. Creating a new column, "max_order" in which to place the results of this aggregation.


In [69]:
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

/var/folders/12/fz0w81fd1mb1hklwgzyg3bx00000gn/T/ipykernel_6445/1681540670.py:1: FutureWarning: The provided callable <function amax at 0x107030ae0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)


In [70]:
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

/var/folders/12/fz0w81fd1mb1hklwgzyg3bx00000gn/T/ipykernel_6445/3647449420.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Loyal customer' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'


In [71]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [72]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [73]:
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

In [74]:
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,orders_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,price_range_loc,busiest_days,busiest_period_of_day,max_order,loyalty_flag
0,2539329,1,1,2,8,NaN,True,196,1,0,both,Soda,77,7,9.0,Mid range product,Regularly busy,Average orders,10,New customer
1,2398795,1,2,3,7,15.0,False,196,1,1,both,Soda,77,7,9.0,Mid range product,Slowest days,Average orders,10,New customer
2,473747,1,3,3,12,21.0,False,196,1,1,both,Soda,77,7,9.0,Mid range product,Slowest days,Most orders,10,New customer
3,2254736,1,4,4,7,29.0,False,196,1,1,both,Soda,77,7,9.0,Mid range product,Slowest days,Average orders,10,New customer
4,431534,1,5,4,15,28.0,False,196,1,1,both,Soda,77,7,9.0,Mid range product,Slowest days,Most orders,10,New customer


## 5. Use Loyalty Flag and check the basic statistics of the product prices for  each loyalty customer
What you're trying to determine is whether the prices of products purchased by loyal customers differ from those purchased by regular or new customers

In [75]:
ords_prods_merge.groupby('loyalty_flag').agg({'prices':['mean','min','max']})

prices           
                      mean  min   max
loyalty_flag                         
Loyal customer    7.773575  1.0  25.0
New customer      7.801206  1.0  25.0
Regular customer  7.798262  1.0  25.0

### noteable - loyal customers purchase items of lower average price than new and regular customers.

UPDATE: with data cleaning removing outliers, price average has lowered and leveled out generally compared to before. Still, loyal customers slightly lower spending per item.

## 6. Create a spending flag for each user based on average price across all their orders using the following criteria:
- if the mean of the prices of products purchased by a user is lower than 10, then flag them as a "Low spender"
- if higher than 10, flag them as a "high spender"

In [76]:
ords_prods_merge['avg_prod_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

/var/folders/12/fz0w81fd1mb1hklwgzyg3bx00000gn/T/ipykernel_6445/3037767957.py:1: FutureWarning: The provided callable <function mean at 0x107031800> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ords_prods_merge['avg_prod_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)


In [77]:
ords_prods_merge.loc[ords_prods_merge['avg_prod_price']< 10, 'spending_type'] = 'Low spender'

/var/folders/12/fz0w81fd1mb1hklwgzyg3bx00000gn/T/ipykernel_6445/1929718489.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Low spender' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ords_prods_merge.loc[ords_prods_merge['avg_prod_price']< 10, 'spending_type'] = 'Low spender'


In [78]:
ords_prods_merge.loc[ords_prods_merge['avg_prod_price']>= 10, 'spending_type'] = 'High spender'

In [79]:
ords_prods_merge['spending_type'].value_counts(dropna = False)

spending_type
Low spender     32285131
High spender      119728
Name: count, dtype: int64

### With our new spending_type flag, we've created two categories of spenders for which to target in our marketing campaigns

## 7. Create order frequency flag that marks the regularity of a user's ordering behavior according to the median days in the "days_since_prior_order" column. The criteria for the flag should be as follows:
- If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”
- If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”
- If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”

In [80]:
ords_prods_merge['ordering_behavior'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

/var/folders/12/fz0w81fd1mb1hklwgzyg3bx00000gn/T/ipykernel_6445/3035388166.py:1: FutureWarning: The provided callable <function median at 0x10716bb00> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  ords_prods_merge['ordering_behavior'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)


In [81]:
ords_prods_merge.loc[ords_prods_merge['ordering_behavior'] > 20, 'frequency_flag'] = 'Non-frequent_customer'

/var/folders/12/fz0w81fd1mb1hklwgzyg3bx00000gn/T/ipykernel_6445/1665527913.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Non-frequent_customer' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ords_prods_merge.loc[ords_prods_merge['ordering_behavior'] > 20, 'frequency_flag'] = 'Non-frequent_customer'


In [82]:
ords_prods_merge.loc[(ords_prods_merge['days_since_prior_order'] > 10) & (ords_prods_merge['ordering_behavior'] <=20.0), 'frequency_flag'] = 'Regular_customer'

In [83]:
ords_prods_merge.loc[ords_prods_merge['ordering_behavior']<= 10, 'frequency_flag'] = 'Frequent customer'

In [84]:
ords_prods_merge['frequency_flag'].value_counts(dropna = False)

frequency_flag
Frequent customer        21559853
Regular_customer          4486672
Non-frequent_customer     3636437
NaN                       2721897
Name: count, dtype: int64

In [85]:
ords_prods_merge.shape

# checking how the table looks now.

(32404859, 24)

In [89]:
ords_prods_merge.columns

Index(['order_id', 'user_id', 'order_number', 'orders_day_of_week',
       'orders_hour_of_day', 'days_since_prior_order', 'first_order',
       'product_id', 'add_to_cart_order', 'reordered', 'product_name',
       'aisle_id', 'department_id', 'prices', 'price_range_loc',
       'busiest_days', 'busiest_period_of_day', 'max_order', 'loyalty_flag',
       'avg_prod_price', 'spending_type', 'ordering_behavior',
       'frequency_flag'],
      dtype='object')

In [88]:
# dropping '_merge' column

ords_prods_merge = ords_prods_merge.drop(columns = '_merge')

## 8. Ensure clean and structured notebook, well commented code.
Could probably do to comment more but I think my processes were straightforward.

## 9. Export dataframe as pickle file and store in "Prepared Data" folder

In [ ]:
ords_prods_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge_48.pkl'))